In [0]:
import os
os.chdir('drive/My Drive/Colab Notebooks/Dacon/AIFrenz_Season2')


In [2]:
pwd

'/content/drive/My Drive/Colab Notebooks/Dacon/AIFrenz_Season2'

In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split , KFold

In [0]:
np.random.seed(777)
tf.set_random_seed(777)

In [0]:
WIDTH = 40
HEIGHT = 40
CHANNEL = 9
IMAGE_SIZE = [WIDTH,HEIGHT,CHANNEL]
numOutputClass  = 1

file_path = './Data'
output_path = './Data/ckpt/output'
pred_path = './Data/ckpt/prediction'

NUM_TRAIN = 76345
NUM_TEST = 2416
BATCH_SIZE = 512
TOTAL_BATCH = int(NUM_TRAIN/BATCH_SIZE)
EPOCHS = 5 

In [0]:
# train_path = './data/dataset/'      ##### path 수정
# image_path = 'train_images.npy'     ##### path 수정
# data_path = 'train_data.npy'        ##### path 수정
# label_path = 'train_labels.npy'     ##### path 수정

# train_image = np.load(train_path+image_path)
# #train_data = np.load(train_path+data_path)
# train_label = np.load(train_path+label_path)
# #train_all = np.concatenate([train_image,train_label],axis=3)

In [0]:
path = './Data/' 
# train_data = np.load(path+'train_imageDataGenerator.npy')
train_data = np.load(path+'train_refined.npy')


In [9]:
train_data.shape

(75917, 40, 40, 15)

In [0]:
#train_images = np.load(path+'./train_images.npy')
#train_labels = np.load(path+'./train_labels.npy')


In [0]:
#train_data = np.concatenate((train_images, train_labels), axis=3)
max_val = train_data.max(axis=(0, 1, 2))
min_val = train_data.min(axis=(0, 1, 2))
train_data = (train_data - min_val) / (max_val - min_val)

In [0]:
train_images = train_data[:,:,:,:9]
train_labels = train_data[:,:,:,-1]
del train_data

In [0]:
train_labels = train_labels.reshape([-1,40,40,1])
train_labels.shape


In [0]:
# 저장된 test 데이터 읽어오기
test_images = np.load('./Data/test_refined.npy')
#test_data = np.load('./Data/test_data.npy')
#del test_data

In [0]:
test_images.shape

In [0]:
#test_image = test_images[:,:,:,:10]

In [0]:
x_train, x_test, y_train, y_test = train_test_split(train_images, train_labels, test_size=0.3, random_state=7777)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [0]:
# 0보다 작은 값이 포함된 데이터 삭제
y_train_ = y_train.reshape(-1, y_train.shape[1]*y_train.shape[2])

x_train = np.delete(x_train, np.where(y_train_ < 0)[0], axis=0)
y_train = np.delete(y_train, np.where(y_train_ < 0)[0], axis=0)
y_train = y_train.reshape(-1, x_train.shape[1], x_train.shape[2],1)
y_test = y_test.reshape(-1, y_test.shape[1], y_test.shape[2],1)

y_train_ = np.delete(y_train_, np.where(y_train_ < 0)[0], axis=0)

# 강수량이 50이상인 데이터만 선택
x_train = x_train[np.sum(y_train_ >= 0)]
y_train = y_train[np.sum(y_train_ >= 50)]

x_train.shape, y_train.shape

In [0]:
#train_label_fit = train_label.reshape(-1,1600)

In [0]:
# def trainGenerator():
#     for i in range(len(train_label)):
#         Xdata = train_image[i,:,:,:]
#         ydata = train_label[i,:,:,:]
#         ydata = ydata.reshape([-1,40*40*1])

#         yield (Xdata , ydata)

# train_dataset = tf.data.Dataset.from_generator(trainGenerator,(tf.float32,tf.float32),(tf.TensorShape([40,40,9]),tf.TensorShape([None,40*40*1])))    ##### shape 수정

# train_dataset = train_dataset.repeat(EPOCHS).batch(BATCH_SIZE).prefetch(1)

# 평가 모델

In [0]:
from sklearn.metrics import f1_score

def mae(y_true, y_pred) :
    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    
    y_pred = y_pred.reshape(1, -1)[0]
    
    over_threshold = y_true >= 0.1
    
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))

def fscore(y_true, y_pred):
    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    
    y_pred = y_pred.reshape(1, -1)[0]
    
    remove_NAs = y_true >= 0
    
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    
    return(f1_score(y_true, y_pred))

def maeOverFscore(y_true, y_pred):
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)

def fscore_keras(y_true, y_pred):
    score = tf.py_function(func=fscore, inp=[y_true, y_pred], Tout=tf.float32, name='fscore_keras')
    return score

def maeOverFscore_keras(y_true, y_pred):
    score = tf.py_function(func=maeOverFscore, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_mse') 
    return score

In [0]:
def mae_over_fscore(y_true, y_pred):
    '''
    y_true: sample_submission.csv 형태의 실제 값
    y_pred: sample_submission.csv 형태의 예측 값
    '''

    y_true = np.array(y_true)
    y_true = y_true.reshape(1, -1)[0]  
    
    y_pred = np.array(y_pred)
    y_pred = y_pred.reshape(1, -1)[0]
    
    # 실제값이 0.1 이상인 픽셀의 위치 확인
    IsGreaterThanEqualTo_PointOne = y_true >= 0.1
    
    # 실제 값에 결측값이 없는 픽셀의 위치 확인 
    IsNotMissing = y_true >= 0
    
    # mae 계산
    mae = np.mean(np.abs(y_true[IsGreaterThanEqualTo_PointOne] - y_pred[IsGreaterThanEqualTo_PointOne]))
    
    # f1_score 계산 위해, 실제값에 결측값이 없는 픽셀에 대해 1과 0으로 값 변환
    y_true = np.where(y_true[IsNotMissing] >= 0.1, 1, 0)
    
    y_pred = np.where(y_pred[IsNotMissing] >= 0.1, 1, 0)
    
    # f1_score 계산    
    f_score = f1_score(y_true, y_pred) 
    
    # f1_score가 0일 나올 경우를 대비하여 소량의 값 (1e-07) 추가 
    return mae / (f_score + 1e-07) 

In [0]:
def conv2d(inputs,filter_size):
    
    conv = Conv2D(filters=filter_size,kernel_size=3,strides=(1,1),padding='same', activation= None, use_bias=True,
                  kernel_initializer='he_uniform')(inputs)
    conv = BatchNormalization()(conv)
    conv = ReLU()(conv)
    
    return conv

def maxpool(conv):
    conv = MaxPool2D(pool_size=(2,2),strides= None, padding='valid')(conv)
    return conv

def deconv2d(conv,filter_size):
    conv = Conv2DTranspose(filters=filter_size,kernel_size=(3,3),strides=(2,2),padding='same',dilation_rate=(1,1),
                           activation=None, use_bias=True,kernel_initializer='he_uniform')(conv)

    return conv

def concat(deconv,conv):
    concat = Concatenate(axis=3)([deconv,conv])

    return concat

def Contracting_Path(inputs,layers=3,base_feature=32):
    dw_conv_list = []
    feature = base_feature

    # Contraction Path
    ## Conv Layer 1
    conv = conv2d(inputs,base_feature)
    conv = conv2d(inputs,base_feature)
    dw_conv_list.append(conv)

    conv = maxpool(conv)

    # Conv Layer 2
    for layer in range(layers-2):
        feature = feature * 2
        conv = conv2d(conv,feature)
        conv = conv2d(conv,feature)
        dw_conv_list.append(conv)

        conv = maxpool(conv)

    ## Conv Layer 3
    feature = feature * 2
    conv = conv2d(conv,feature)
    conv = conv2d(conv,feature)

    return conv , dw_conv_list

def Expanding_Path(conv,dw_conv_list):
    
    while len(dw_conv_list) != 0:
        dw_conv = dw_conv_list.pop()
        feature = dw_conv.shape[3]

        deconv = deconv2d(conv,feature)
        conv_cat = concat(deconv,dw_conv)

        conv = conv2d(conv_cat , (conv_cat.shape[3]//2))
        conv = conv2d(conv , (conv.shape[3]))

    return conv

In [0]:
def Unet():
    inputs = Input([40,40,9])

    conv , dw_conv_list = Contracting_Path(inputs,layers=3,base_feature=32)

    conv = Expanding_Path(conv,dw_conv_list)

    output = conv2d(conv,1)

    model = Model(inputs=inputs,outputs=output)
    model.compile(loss="mae", optimizer="adam", metrics=[fscore_keras,maeOverFscore_keras])

    return model

In [0]:
model = Unet()
model.summary()

# Training


In [0]:
# ReduceLROnPlateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',patience=3) # factor=0.05
# EarlyStop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=7,restore_best_weights=True)

# callbackList = [EarlyStop,ReduceLROnPlateau]

# hist = model.fit(x_train,y_train,epochs=50,batch_size=512,validation_split=0.25, callbacks = callbackList, verbose=1)

In [0]:
def train_model(x_data, y_data, k, s):
    k_fold = KFold(n_splits=k, shuffle=True, random_state=777)
    
    model_number = 0
    for train_idx, val_idx in k_fold.split(x_data):
        if model_number == s:
            x_train, y_train = x_data[train_idx], y_data[train_idx]
            x_val, y_val = x_data[val_idx], y_data[val_idx]

            model = Unet()

            ReduceLROnPlateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_maeOverFscore_keras',patience=3) # factor=0.05
            EarlyStop = tf.keras.callbacks.EarlyStopping(monitor='val_maeOverFscore_keras',patience=3,restore_best_weights=True)
            ModelCkpt = tf.keras.callbacks.ModelCheckpoint(filepath='./Data/ckpt/Unet_' + str(model_number)+'.h5',monitor='val_maeOverFscore_keras',save_best_only=True)
            callbackList = [EarlyStop,ReduceLROnPlateau]

            model.fit(x_train, y_train, epochs=50, batch_size=128, validation_data=(x_val, y_val), callbacks=callbackList)
        
        model_number += 1

In [0]:
k = 5
models = []
preds = []

train_model(x_train, y_train, k=k, s=4)

for n in range(k):
    model = load_model('./Data/ckpt/Unet_' + str(n) + '.h5', custom_objects = {'score': score,'fscore_keras': fscore_keras})
    models.append(model)

for model in models:
    preds.append(model.predict(x_test))
    print(mae_over_fscore(y_test, preds[-1]))

pred = sum(preds)/len(preds)
print(mae_over_fscore(y_test, pred))

In [0]:
model.evaluate(x_test,y_test,verbose=1)

In [0]:
prediction = model.predict(test_image)

In [0]:
# submission = pd.read_csv('data/sample_submission.csv')
# submission.iloc[:,1:] = prediction.reshape([-1,1600])
# submission.to_csv('data/U_Net_ver1_1.csv', index=False)

In [0]:
# model.save('U_Net_200520_ver_1_1.h5')